In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.utils.data as Data

In [2]:
# Hyper parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 64
TIME_STEMP = 28         # rnn time stem/ image height
INPUT_SIZE =28          # rnn input size / image width
LR =0.01                # learning rate
DOWNLOAD_MNIST = False  # set to True if haven't download the data

train_data = dsets.MNIST(
    root ='/Users/chauncey_chen/Downloads/mnist',
    train= True,
    transform = transforms.ToTensor(),               # (0, 1)       (0-255)
    download = DOWNLOAD_MNIST
)

In [3]:
test_data=dsets.MNIST(root='/Users/chauncey_chen/Downloads/mnist', train=False)
# Data Loader for easy mini-batch return in training
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_x = test_data.test_data.type(torch.FloatTensor)[:2000]/255.   # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy()[:2000]    # covert to numpy array

In [4]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.LSTM(
            input_size = INPUT_SIZE,
            hidden_size = 64,
            num_layers = 1,   #
            batch_first= True
        )
        self.out = nn.Linear(64, 10)
    
    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)   # x (batch,time_step,input_size)
        out = self.out(r_out[:, -1, :])   #(batch, time step, input)
        return out

In [5]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
optimizer = torch.optim.Adam(rnn.parameters(), lr = LR)   #optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                         # the target label if not one-hotted

In [7]:
# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):        # gives batch data
        b_x = b_x.view(-1, 28, 28)              # reshape x to (batch, time_step, input_size)

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)


# print 10 predictions from test data
test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y, 'prediction number')
print(test_y[:10], 'real number')
"""
[7 2 1 0 4 1 4 9 5 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number
"""

Epoch:  0 | train loss: 2.3112 | test accuracy: 0.17
Epoch:  0 | train loss: 1.0790 | test accuracy: 0.54
Epoch:  0 | train loss: 0.7720 | test accuracy: 0.73
Epoch:  0 | train loss: 0.6939 | test accuracy: 0.80
Epoch:  0 | train loss: 0.2774 | test accuracy: 0.86
Epoch:  0 | train loss: 0.1866 | test accuracy: 0.89
Epoch:  0 | train loss: 0.3430 | test accuracy: 0.90
Epoch:  0 | train loss: 0.3215 | test accuracy: 0.89
Epoch:  0 | train loss: 0.3015 | test accuracy: 0.91
Epoch:  0 | train loss: 0.2314 | test accuracy: 0.91
Epoch:  0 | train loss: 0.1320 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1118 | test accuracy: 0.94
Epoch:  0 | train loss: 0.2262 | test accuracy: 0.94
Epoch:  0 | train loss: 0.2138 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2427 | test accuracy: 0.91
Epoch:  0 | train loss: 0.1158 | test accuracy: 0.95
Epoch:  0 | train loss: 0.0989 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1504 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2186 | test accuracy

'\n[7 2 1 0 4 1 4 9 5 9] prediction number\n[7 2 1 0 4 1 4 9 5 9] real number\n'